## ETL_PurchasesFINAL12312016

In [1]:
# --- 1. IMPORTAR LIBRERÍAS ---
import pandas as pd
import numpy as np
import re
import matplotlib as plt
import seaborn as sn

In [3]:
# --- 2. CARGA DE DATOS ---
df = pd.read_csv(r"C:\Users\danyf\Desktop\Proyecto_Final_Grupo_2\PurchasesFINAL12312016.csv", low_memory=False, encoding='utf-8')
print("Datos cargados correctamente")
print("Dimensiones:", df.shape)

Datos cargados correctamente
Dimensiones: (2372474, 16)


In [4]:
# --- 3. NORMALIZAR NOMBRES DE COLUMNAS (snake_case) ---
def to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    s3 = s2.replace('-', '_').replace(' ', '_')
    return re.sub(r'__+', '_', s3).strip('_').lower()

df.columns = [to_snake_case(col) for col in df.columns]

In [5]:
# Verificamos los primeros registros
df.head()

,inventory_id,store,brand,description,size,vendor_number,vendor_name,po_number,po_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.32,5,106.60,1


In [6]:
# ======================================================
# 🔠 Normalización de texto: columna Description a mayúsculas
# ======================================================

if "description" in df.columns:
    df["description"] = df["description"].astype(str).str.upper()

print(df["description"].head())

0       TEQUILA OCHO PLATA FRESNO
1    TGI FRIDAYS ULTIMTE MUDSLIDE
2    TGI FRIDAYS LONG ISLAND ICED
3    TGI FRIDAYS ULTIMTE MUDSLIDE
4       GLENDALOUGH DOUBLE BARREL
Name: description, dtype: object


In [7]:
# --- 4. CONVERSIÓN DE TIPOS DE DATOS ---
# Numéricos
for col in ['purchase_price', 'quantity', 'dollars']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
# Convertir columnas de texto a string
text_cols = ["description", "vendor_name"]
df[text_cols] = df[text_cols].astype("string")

In [9]:
# Fechas → formato datetime
date_cols = [c for c in ['po_date', 'receiving_date', 'invoice_date', 'pay_date'] if c in df.columns]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [10]:
print(df.dtypes)

inventory_id              object
store                      int64
brand                      int64
description       string[python]
size                      object
vendor_number              int64
vendor_name       string[python]
po_number                  int64
po_date           datetime64[ns]
receiving_date    datetime64[ns]
invoice_date      datetime64[ns]
pay_date          datetime64[ns]
purchase_price           float64
quantity                   int64
dollars                  float64
classification             int64
dtype: object


In [11]:
# Chequeo de nulos y duplicados
def revisar_datos(df):
    print("🔍 Revisión inicial de los datos:")
    print(f"Forma del DataFrame (filas, columnas): {df.shape}\n")
    
    print("Valores faltantes por columna:\n", df.isnull().sum())
    print("\nTotal de filas duplicadas:", df.duplicated().sum())

In [12]:
revisar_datos(df)

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (2372474, 16)

Valores faltantes por columna:
 inventory_id      0
store             0
brand             0
description       0
size              3
vendor_number     0
vendor_name       0
po_number         0
po_date           0
receiving_date    0
invoice_date      0
pay_date          0
purchase_price    0
quantity          0
dollars           0
classification    0
dtype: int64

Total de filas duplicadas: 0


In [13]:
# reemplazar todos los valores nulos (NaN) por el valor 750
df['size'].fillna(750, inplace=True)

C:\Users\danyf\AppData\Local\Temp\ipykernel_23164\1220054052.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['size'].fillna(750, inplace=True)


In [14]:
# Verificamos que ya no hay datos faltantes
revisar_datos(df)

🔍 Revisión inicial de los datos:
Forma del DataFrame (filas, columnas): (2372474, 16)

Valores faltantes por columna:
 inventory_id      0
store             0
brand             0
description       0
size              0
vendor_number     0
vendor_name       0
po_number         0
po_date           0
receiving_date    0
invoice_date      0
pay_date          0
purchase_price    0
quantity          0
dollars           0
classification    0
dtype: int64

Total de filas duplicadas: 0


In [15]:
# --- 5. TRATAMIENTO DE LA COLUMNA 'size' ---
# Normalizar texto
s = (
    df['size']
    .astype('string')
    .str.lower()
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
    .str.replace(r'pk\.?', ' pk', regex=True)
    .str.replace(r'pack', ' pk', regex=True)
    .str.replace(r'packs', ' pk', regex=True)
    .str.replace(r'liter', 'l', regex=True)
)

# Patrón regex para capturar número, unidad y cantidad
pat = (
    r'(?P<num>\d+(?:\.\d+)?)\s*'
    r'(?P<unit>ml|l|oz|gal)?'
    r'(?:\s*(?P<qty>\d+)\s*pk)?'
    r'(?:\s*\+\s*(?P<extra>\d+)\s*/)?'
    r'(?:\s*/\s*(?P<split>\d+)\s*)?'
)
m = s.str.extract(pat)

# --- Función para convertir todo a mililitros ---
def to_ml(row, original_value):
    num, unit, split = row['num'], row['unit'], row['split']

    # “liter” escrito sin número → 1 L = 1000 mL
    if isinstance(original_value, str) and original_value.strip().lower() == 'liter':
        return 1000.0

    if pd.isna(num):
        return np.nan

    try:
        num = float(num)
    except:
        return np.nan

    if pd.notna(split):
        try:
            num = num / float(split)
        except:
            pass

    if pd.isna(unit):
        return np.nan
    unit = unit.lower()

    # Conversión a mL
    if unit == 'ml':
        return num
    elif unit == 'l':
        return num * 1000
    elif unit == 'oz':
        return num * 29.5735
    elif unit == 'gal':
        return num * 3785.41
    else:
        return np.nan

# Aplicar conversión
df['size_ml'] = [
    to_ml(row, orig) for (_, row), orig in zip(m.iterrows(), df['size'])
]
df['size_ml'] = pd.to_numeric(df['size_ml'], errors='coerce').round(0).astype('Int64')

# Calcular cantidad (packs)
qty_pk = pd.to_numeric(m['qty'], errors='coerce').astype('Int64')
extra = pd.to_numeric(m['extra'], errors='coerce').astype('Int64')

df['size_qty'] = qty_pk.combine_first(extra)
df['size_qty'] = df['size_qty'].fillna(1).astype('Int64')

# Si no hay volumen, dejar cantidad vacía
df.loc[df['size_ml'].isna(), 'size_qty'] = pd.NA

In [16]:
# revisamos algunos registros, confirmamos que se agregaron correctamente las columnas
df.head()

,inventory_id,store,brand,description,size,vendor_number,vendor_name,po_number,po_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,size_ml,size_qty
0,69_MOUNTMEND_8412,69,8412,TEQUILA OCHO PLATA FRESNO,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.71,6,214.26,1,750,1
1,30_CULCHETH_5255,30,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,4,37.40,1,1750,1
2,34_PITMERDEN_5215,34,5215,TGI FRIDAYS LONG ISLAND ICED,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.41,5,47.05,1,1750,1
3,1_HARDERSFIELD_5255,1,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,6,56.10,1,1750,1
4,76_DONCASTER_2034,76,2034,GLENDALOUGH DOUBLE BARREL,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.32,5,106.60,1,750,1


In [17]:
# ======================================================
# 🧭 Enriquecimiento de datos logísticos y de pago
# ======================================================

# Asegurarse de que las columnas de fecha estén en formato datetime
for col in ["po_date", "receiving_date", "invoice_date", "pay_date"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Calcular tiempos en días entre etapas del proceso
df["lead_time_days"] = (df["receiving_date"] - df["po_date"]).dt.days
df["processing_time_days"] = (df["invoice_date"] - df["receiving_date"]).dt.days
df["payment_cycle_days"] = (df["pay_date"] - df["invoice_date"]).dt.days

# Verificar secuencia temporal
invalid_times = (
    (df["lead_time_days"] < 0).sum() +
    (df["processing_time_days"] < 0).sum() +
    (df["payment_cycle_days"] < 0).sum()
)
print(f"⏱️ Fechas con secuencia incorrecta: {invalid_times}")

# Estadísticas descriptivas de los nuevos campos
df_times_summary = df[["lead_time_days", "processing_time_days", "payment_cycle_days"]].describe().round(2)
display(df_times_summary)

# Guardar resumen de tiempos logísticos
df_times_summary.to_csv("logistics_time_summary.csv", index=True)

⏱️ Fechas con secuencia incorrecta: 0


,lead_time_days,processing_time_days,payment_cycle_days
count,2372474.00,2372474.00,2372474.00
mean,7.62,8.87,35.66
std,2.21,2.98,6.06
min,3.00,1.00,23.00
25%,6.00,7.00,31.00
50%,8.00,9.00,36.00
75%,9.00,11.00,41.00
max,14.00,15.00,48.00


In [18]:
# Verificamos que se agregaran las columnas
df.head()

,inventory_id,store,brand,description,size,vendor_number,vendor_name,po_number,po_date,receiving_date,...,pay_date,purchase_price,quantity,dollars,classification,size_ml,size_qty,lead_time_days,processing_time_days,payment_cycle_days
0,69_MOUNTMEND_8412,69,8412,TEQUILA OCHO PLATA FRESNO,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,...,2016-02-16,35.71,6,214.26,1,750,1,12,2,43
1,30_CULCHETH_5255,30,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,...,2016-02-21,9.35,4,37.40,1,1750,1,10,6,45
2,34_PITMERDEN_5215,34,5215,TGI FRIDAYS LONG ISLAND ICED,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,...,2016-02-21,9.41,5,47.05,1,1750,1,11,5,45
3,1_HARDERSFIELD_5255,1,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,...,2016-02-21,9.35,6,56.10,1,1750,1,10,6,45
4,76_DONCASTER_2034,76,2034,GLENDALOUGH DOUBLE BARREL,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,...,2016-02-16,21.32,5,106.60,1,750,1,9,7,38


In [19]:
# --- 7. DETECCIÓN Y TRATAMIENTO DE OUTLIERS ---
# (Solo identificamos, no eliminamos aún)
def detectar_outliers(col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    return len(outliers), lower, upper

for col in ['purchase_price', 'quantity', 'dollars']:
    if col in df.columns:
        n_out, low, up = detectar_outliers(col)
        print(f"{col}: {n_out} outliers detectados (fuera de [{low:.2f}, {up:.2f}])")

purchase_price: 137317 outliers detectados (fuera de [-6.44, 27.05])
quantity: 284431 outliers detectados (fuera de [-3.00, 21.00])
dollars: 191797 outliers detectados (fuera de [-87.63, 277.41])


In [28]:
#No elimino los outliers, pero los etiqueto en una nueva columna. 
#Así puedes decidir más adelante si excluirlos solo en ciertos análisis.

for col in['purchase_price', 'quantity', 'dollars'] :
    if col in df.columns:
        # Calcular límites con IQR
        q1, q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr

        # Crear columna booleana que indique si el valor es outlier
        df[f"{col}_outlier"] = ~df[col].between(lower, upper)

In [32]:
# Verificamos las nuevas columnas
df.head()

,inventory_id,store,brand,description,size,vendor_number,vendor_name,po_number,po_date,receiving_date,...,dollars,classification,size_ml,size_qty,lead_time_days,processing_time_days,payment_cycle_days,purchase_price_outlier,quantity_outlier,dollars_outlier
0,69_MOUNTMEND_8412,69,8412,TEQUILA OCHO PLATA FRESNO,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,...,214.26,1,750,1,12,2,43,True,False,False
1,30_CULCHETH_5255,30,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,...,37.40,1,1750,1,10,6,45,False,False,False
2,34_PITMERDEN_5215,34,5215,TGI FRIDAYS LONG ISLAND ICED,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,...,47.05,1,1750,1,11,5,45,False,False,False
3,1_HARDERSFIELD_5255,1,5255,TGI FRIDAYS ULTIMTE MUDSLIDE,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,...,56.10,1,1750,1,10,6,45,False,False,False
4,76_DONCASTER_2034,76,2034,GLENDALOUGH DOUBLE BARREL,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,...,106.60,1,750,1,9,7,38,False,False,False


In [31]:
# 🔟 Guardar el archivo limpio
df.to_csv("PurchasesFINAL_limpio.csv", index=False)
